In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [18]:
# Should be used only one for cookie storage.

def login_to_linkedin(driver):
    driver.get("https://www.linkedin.com/login")
    username_input = driver.find_element(By.ID, "username")
    password_input = driver.find_element(By.ID, "password")
    
    username_input.send_keys(email)
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)  # Press ENTER to submit form

def wait_for_2fa(driver):
    # Adjust the selector to identify an element present after 2FA completion
    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'scaffold-layout__main'))
    )
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')
login_to_linkedin(driver)
wait_for_2fa(driver)

cookies = driver.get_cookies()
with open('cookies.pkl', 'wb') as cookies_file:
    pickle.dump(cookies, cookies_file)

In [19]:
def get_span_value_by_class_name(driver, class_name):
    values = []
    elements = driver.find_elements(By.CLASS_NAME, class_name)
    for element in elements:
        anchor = element.find_element(By.TAG_NAME, 'a')
        if anchor:
            spans = anchor.find_elements(By.TAG_NAME, 'span')
            for span in spans:
                text = span.text
                if 'profile' not in text:
                    values.append(text)
    return values

def delayed_loop(driver):
    with open('names.txt', 'w') as file:
        for i in range(5):
            sleep(5)  # Wait for 5 seconds
            scroll_to_end(driver)
            values = get_span_value_by_class_name(driver, 'entity-result__title-text')
            file.write("\n".join(values))
            file.write("\n")
            driver.find_element(By.CLASS_NAME, 'artdeco-pagination__button--next').click()
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if lines:
        lines = lines[:-1]
    with open(file_path, 'w') as file:
        file.writelines(lines)
    

def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(5)  # Wait for page elements to load after scrolling

driver = webdriver.Chrome()
driver.get("https://www.linkedin.com")

with open('cookies.pkl', 'rb') as cookies_file:
    cookies = pickle.load(cookies_file)
    for cookie in cookies:
        driver.add_cookie(cookie)

driver.get('https://www.linkedin.com/search/results/people/?keywords='+company+'&origin=SWITCH_SEARCH_VERTICAL')  # Adjust URL as needed
scroll_to_end(driver)
delayed_loop(driver)

driver.quit()  # Close the browser

def generate_email_template(first_name, last_name, company_name):
    subject = f"""Exploring Insights and Guidance in {company_name}"""
    body = f"""
Dear {first_name},
    
I hope this message finds you well. My name is Pradhyuman Yadav, a Master's student in Computer Science at George Washington University with a background in software engineering at Optum. My journey has been enriched with experiences in developing enterprise applications, managing Docker and Kubernetes deployments, and contributing to projects that enhance system efficiency and responsiveness.
    
Your work at {company_name} has caught my attention and I admire what all {company_name} has been doing these days and the presence and reputation it has gained, and I am eager to learn more about your experiences and insights in this field.
    
Would you be open to a brief, 10-minute conversation where we could discuss your journey, the challenges you've faced, and the lessons you've learned along the way? Your guidance and mentorship would be invaluable to me as I navigate my path and explore how I can contribute to fields that excite me.
    
Thank you for considering my request. I look forward to the possibility of speaking with you.
    
Warm regards,
Pradhyuman Yadav
Mobile <+1 571-259-4892>
Linkedin <https://www.linkedin.com/in/pradhyuman-yadav>
Github <https://github.com/yadavpradhyuman>"""
    return subject, body

def send_email(recipient_email, subject, body):
    # Set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)  # Use your SMTP server details
    s.starttls()
    s.login(gmail_Email,gmail_Password)

    # Create the email
    msg = MIMEMultipart()
    msg['From'] = "usa.pradhyuman@gmail.com"
    msg['To'] = recipient_email
    msg['Subject'] = subject

    with open("/Users/pradhyumanyadav/Desktop/OOD_Project/Email Generator/src/Pradhyuman_Resume_Professional.pdf", 'rb') as attachment_file:
        part = MIMEApplication(attachment_file.read(), Name='Resume_Pradhyuman.pdf')
        part['Content-Disposition'] = 'attachment; filename="Resume_Pradhyuman.pdf"'
        msg.attach(part)

    msg.attach(MIMEText(body, 'plain'))

    # Send the email
    s.send_message(msg)
    print("Email sent to:"+ recipient_email)
    s.quit()


def generate_email_addresses(first_name, last_name, company_name):
    # Define the list of email formats as a function that takes first and last names
    def email_formats(first_name, last_name):
        initials = f"{first_name[0]}{last_name[0]}"
        return [
            f"{first_name}{last_name}@{company_name}",
            f"{first_name}.{last_name}@{company_name}",
            f"{first_name}_{last_name}@{company_name}",
            f"{first_name[0]}{last_name}@{company_name}",
            f"{first_name[0]}.{last_name}@{company_name}",
            f"{first_name[0]}_{last_name}@{company_name}",
            f"{first_name}{last_name[0]}@{company_name}",
            f"{last_name}{first_name}@{company_name}",
            f"{last_name}.{first_name}@{company_name}",
            f"{last_name}_{first_name}@{company_name}",
            f"{last_name[0]}{first_name}@{company_name}",
            f"{last_name[0]}.{first_name}@{company_name}",
            f"{last_name[0]}_{first_name}@{company_name}",
            f"{first_name}-{last_name}@{company_name}",
            f"{first_name}{last_name}1@{company_name}",
            f"{first_name}.{last_name}2@{company_name}",
            f"{first_name}_{last_name}3@{company_name}",
            f"{first_name[0]}{last_name}4@{company_name}",
            f"{first_name[0]}.{last_name}5@{company_name}",
            f"{first_name[0]}_{last_name}6@{company_name}",
            f"{first_name}{last_name[0]}7@{company_name}",
            f"{last_name}{first_name}8@{company_name}",
            f"{last_name}.{first_name}9@{company_name}",
            f"{last_name}_{first_name}10@{company_name}",
            f"{last_name[0]}{first_name}11@{company_name}",
            f"{last_name[0]}.{first_name}12@{company_name}",
            f"{last_name[0]}_{first_name}13@{company_name}",
            f"{first_name}-{last_name}14@{company_name}",
            # The pattern continues, incrementing numbers or changing separators as necessary
            # This list is illustrative and repeats patterns with different separators or numbers for demonstration
            # In a real-world scenario, the format list would be tailored to the organization's actual email format policies
        ]

    # email_list = []
    emails = email_formats(first_name.lower(), last_name.lower())
    # email_list.append(emails)

    return emails

# Example usage
file_path = '/Users/pradhyumanyadav/Downloads/EmailSender/names.txt'  # Replace with the actual file path
# company_name = company+'.com'  # Replace with the actual company domain
# emails = generate_email_addresses(file_path, company_name)

# with open("output.txt", 'w') as fileOutput:
with open(file_path, 'r') as file:
    for line in file:
        line = line.replace(".", "")
        first_name, *middle_name, last_name = line.strip().split()
        subject, body = generate_email_template(first_name, last_name, company)
        emails = generate_email_addresses(first_name, last_name, company+'.com')
        for email in emails:
            # fileOutput.write("\nEmail = "+email+"\nSubject = "+subject+"\nBody = "+body[0:10])
            send_email(email, subject, body)

Email sent to:shivamgupta@GWU.edu
Email sent to:shivam.gupta@GWU.edu
Email sent to:shivam_gupta@GWU.edu
Email sent to:sgupta@GWU.edu
Email sent to:s.gupta@GWU.edu
Email sent to:s_gupta@GWU.edu
Email sent to:shivamg@GWU.edu
Email sent to:guptashivam@GWU.edu
Email sent to:gupta.shivam@GWU.edu
Email sent to:gupta_shivam@GWU.edu
Email sent to:gshivam@GWU.edu
Email sent to:g.shivam@GWU.edu
Email sent to:g_shivam@GWU.edu
Email sent to:shivam-gupta@GWU.edu
Email sent to:shivamgupta1@GWU.edu
Email sent to:shivam.gupta2@GWU.edu
Email sent to:shivam_gupta3@GWU.edu
Email sent to:sgupta4@GWU.edu
Email sent to:s.gupta5@GWU.edu
Email sent to:s_gupta6@GWU.edu
Email sent to:shivamg7@GWU.edu
Email sent to:guptashivam8@GWU.edu
Email sent to:gupta.shivam9@GWU.edu
Email sent to:gupta_shivam10@GWU.edu
Email sent to:gshivam11@GWU.edu
Email sent to:g.shivam12@GWU.edu
Email sent to:g_shivam13@GWU.edu
Email sent to:shivam-gupta14@GWU.edu
Email sent to:pradhyumanyadav@GWU.edu
Email sent to:pradhyuman.yadav@GWU.